In [1]:
# import dependencies
import zipfile
import os.path
import pandas as pd
import numpy as np


In [15]:
# Define zipped files for processing
# GitHub has 100MB file size limit so had to break them up
directory = 'data'
files = [
    '{}/VCardTransactions-NoName.csv.zip'.format(directory),
    '{}/VCardTransactions-NoName2.csv.zip'.format(directory)
]


# Unzip files if they are not unzipped on local machine
for file in files:
    csvFile = file.replace('.zip','')
    
    if(not os.path.exists(csvFile)):
        print('File DOES NOT exist {} - un-zipping'.format(csvFile))
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(directory)
    else:
        print('File exists {} - NOT un-zipping'.format(csvFile))


# Create combined CSV files

viewFile = '{}/vtrans.csv'.format(directory)
os.makedirs(os.path.dirname(viewFile), exist_ok=True)
if(not os.path.exists(viewFile)):
    print('File NOT exists {} - merging'.format(viewFile))
    with open(files[0].replace('.zip','')) as fp:
        data = fp.read()

    with open(files[1].replace('.zip','')) as fp:
        data2 = fp.read()

    data += "\n"
    data += data2

    with open (viewFile, 'w') as fp:
        fp.write(data)
else:
    print('File exists {} - NOT merging'.format(viewFile))

lotFile = '{}/lot_facts.csv'.format(directory)

File exists data/VCardTransactions-NoName.csv - NOT un-zipping
File exists data/VCardTransactions-NoName2.csv - NOT un-zipping
File exists data/vtrans.csv - NOT merging


In [16]:
# read files into a data frame
vtrans = pd.read_csv(viewFile)
capacity = pd.read_csv(lotFile)


In [46]:
# read lot facts csv file into pandas dataframe
lotdata = pd.read_csv('data/lot_facts.csv')
lotdata = lotdata.sort_values(by='lot', ascending=True)
lotdata = lotdata.reset_index(drop=True)
lotdata.rename(columns = {"lot" : "LotNumber"}, inplace = True) #renamed the column to be able to bring capacity into the vtrans later


In [47]:
lotdata

,LotNumber,capacity
0,2,34
1,3,217
2,4,48
3,6,20
4,7,10
5,8,37
6,9,18
7,10,462
8,11,591
9,12,88


In [18]:
# return vtrans dataframe
vtrans

,tTransID,FullName,CardNumber,LotNumber,EffectiveGroupName,EntranceTime,ExitTime,Sort_Time,BusinessHours,NWHours,UseMoreThan3Hours,HourExceed3Hours,NoEntry,noexit,Consecutive,Overnight
0,22840709,NaN,12777,13,NaN,2022-07-11 12:29:09.493,NaN,2022-07-11 12:29:09.493,NaN,NaN,NaN,NaN,0,1,0,0
1,22840716,NaN,47951,40,NaN,2022-07-11 12:29:09.367,NaN,2022-07-11 12:29:09.367,NaN,NaN,NaN,NaN,0,1,0,0
2,22840719,NaN,58760,3,NaN,2022-07-11 12:29:04.227,NaN,2022-07-11 12:29:04.227,NaN,NaN,NaN,NaN,0,1,0,0
3,22840717,NaN,57369,20,NaN,2022-07-11 12:29:02.680,NaN,2022-07-11 12:29:02.680,NaN,NaN,NaN,NaN,0,1,0,0
4,22840710,NaN,13947,11,NaN,2022-07-11 12:28:53.320,NaN,2022-07-11 12:28:53.320,NaN,NaN,NaN,NaN,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743017,9008592,NaN,22348,70,NW (With Permit),2018-01-01 02:47:54.023,2018-01-01 15:09:22.233,2018-01-01 02:47:54.023,0.0,13.0,0.0,0.0,0,0,0,0
6743018,9007839,NaN,40907,70,NW ONLY,2018-01-01 01:53:11.657,2018-01-01 02:43:54.163,2018-01-01 01:53:11.657,0.0,1.0,0.0,0.0,0,0,0,0
6743019,9007851,NaN,2005,10,Lot 10,2018-01-01 01:33:14.233,2018-01-01 05:03:29.757,2018-01-01 01:33:14.233,0.0,4.0,0.0,0.0,0,0,0,0
6743020,9056422,NaN,23050,70,NW (With Permit),2018-01-01 01:14:17.293,NaN,2018-01-01 01:14:17.293,NaN,NaN,NaN,NaN,0,1,1,0


In [22]:
# breakout vtrans date/time/day of week into individual columns

vtrans['EntranceDate'] = pd.to_datetime(vtrans['EntranceTime']).dt.date
vtrans['EntranceTime'] = pd.to_datetime(vtrans['EntranceTime']).dt.strftime('%H:%M:%S')
vtrans['ExitDate'] = pd.to_datetime(vtrans['ExitTime']).dt.date
vtrans['ExitTime'] = pd.to_datetime(vtrans['ExitTime']).dt.strftime('%H:%M:%S')

# Sort date determined to not be needed - commenting out
# vtrans['SortDate'] = pd.to_datetime(vtrans['Sort_Time']).dt.date
# vtrans['SortTime'] = pd.to_datetime(vtrans['Sort_Time']).dt.strftime('%H:%M:%S')


In [23]:
# breakout vtrans day of week into individual column
vtrans['EntranceDay'] = pd.to_datetime(vtrans['EntranceDate']).dt.day_name()
vtrans['ExitDay'] = pd.to_datetime(vtrans['ExitDate']).dt.day_name()

# Sort date determined to not be needed - commenting out
# vtrans['SortDay'] = pd.to_datetime(vtrans['Sort_Time']).dt.day_name()

In [24]:
# rearrange and organize columns
col_titles = ['tTransID', 'CardNumber', 'LotNumber', 'EffectiveGroupName','EntranceDate', 'EntranceTime', 'EntranceDay',
       'ExitDate', 'ExitTime', 'ExitDay', 'NoEntry', 'noexit']

vtrans = vtrans.reindex(columns=col_titles)
vtrans

,tTransID,CardNumber,LotNumber,EffectiveGroupName,EntranceDate,EntranceTime,EntranceDay,ExitDate,ExitTime,ExitDay,NoEntry,noexit
0,22840709,12777,13,NaN,2022-07-11,12:29:09,Monday,NaT,NaN,NaN,0,1
1,22840716,47951,40,NaN,2022-07-11,12:29:09,Monday,NaT,NaN,NaN,0,1
2,22840719,58760,3,NaN,2022-07-11,12:29:04,Monday,NaT,NaN,NaN,0,1
3,22840717,57369,20,NaN,2022-07-11,12:29:02,Monday,NaT,NaN,NaN,0,1
4,22840710,13947,11,NaN,2022-07-11,12:28:53,Monday,NaT,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6743017,9008592,22348,70,NW (With Permit),2018-01-01,02:47:54,Monday,2018-01-01,15:09:22,Monday,0,0
6743018,9007839,40907,70,NW ONLY,2018-01-01,01:53:11,Monday,2018-01-01,02:43:54,Monday,0,0
6743019,9007851,2005,10,Lot 10,2018-01-01,01:33:14,Monday,2018-01-01,05:03:29,Monday,0,0
6743020,9056422,23050,70,NW (With Permit),2018-01-01,01:14:17,Monday,NaT,NaN,NaN,0,1


In [25]:
# getting counts for days of week to verify to_day code worked as expected. 
e_count = vtrans.value_counts("EntranceDay")
x_count = vtrans.value_counts("ExitDay")

# show value counts

e_count
# x_count

EntranceDay
Wednesday    1188984
Tuesday      1188198
Thursday     1159389
Monday       1082119
Friday       1061582
Saturday      533065
Sunday        529685
dtype: int64

In [26]:
# Defining a widget to select a certain lot#

from ipywidgets import widgets, interactive, Layout

w_lot = widgets.Dropdown(
    options = ["All"] + sorted(set(vtrans.LotNumber)),     
    value = "All",
    description = 'Lot #:',
    style = {"description_width": '100px'},
    layout = Layout(width="25%"),
    disabled = False
)

def view(lot):
    if lot == "All":
        dftmp = vtrans
    else:
        dftmp = vtrans[vtrans.LotNumber == lot]
    
    display(dftmp)

i = interactive(view, lot=w_lot)
display(i)

interactive(children=(Dropdown(description='Lot #:', layout=Layout(width='25%'), options=('All', 0, 3, 4, 6, 8…

In [27]:
# Defining a widget to select data for a certain day of the week

from ipywidgets import widgets, interactive, Layout

w_dow = widgets.RadioButtons(
    options = ["All", "Monday", "Tuesday", "Wednesday", "Thursday","Friday","Saturday","Sunday"],     
    value = "All",
    description = 'Day of the week:',
    style = {"description_width": '100px'},
#     layout = Layout(width="25%"),
    disabled = False
)

def view(dow):
    if dow == "All":
        dftmp = vtrans
    else:
        dftmp = vtrans[vtrans.EntranceDay == dow]
    
    display(dftmp)

i = interactive(view, dow=w_dow)
display(i)

interactive(children=(RadioButtons(description='Day of the week:', options=('All', 'Monday', 'Tuesday', 'Wedne…

In [29]:
# Calculating the number of days a car was parked

vtrans['ParkedDays'] = vtrans['ExitDate'] - vtrans['EntranceDate']


In [37]:
# Calculating the number of hours a car was parked

vtrans["EntranceTime"] = pd.to_datetime(vtrans["EntranceTime"])
vtrans["ExitTime"] = pd.to_datetime(vtrans["ExitTime"])
vtrans["EntryHour"] = vtrans["EntranceTime"].dt.hour
vtrans["ExistHour"] = vtrans["ExitTime"].dt.hour
vtrans["ParkedTime"] = vtrans["ExistHour"] - vtrans["EntryHour"]
vtrans

,tTransID,CardNumber,LotNumber,EffectiveGroupName,EntranceDate,EntranceTime,EntranceDay,ExitDate,ExitTime,ExitDay,NoEntry,noexit,ParkedDays,EntryHour,ExistHour,ParkedTime
0,22840709,12777,13,NaN,2022-07-11,2022-07-17 12:29:09,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN
1,22840716,47951,40,NaN,2022-07-11,2022-07-17 12:29:09,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN
2,22840719,58760,3,NaN,2022-07-11,2022-07-17 12:29:04,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN
3,22840717,57369,20,NaN,2022-07-11,2022-07-17 12:29:02,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN
4,22840710,13947,11,NaN,2022-07-11,2022-07-17 12:28:53,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743017,9008592,22348,70,NW (With Permit),2018-01-01,2022-07-17 02:47:54,Monday,2018-01-01,2022-07-17 15:09:22,Monday,0,0,0 days,2,15.0,13.0
6743018,9007839,40907,70,NW ONLY,2018-01-01,2022-07-17 01:53:11,Monday,2018-01-01,2022-07-17 02:43:54,Monday,0,0,0 days,1,2.0,1.0
6743019,9007851,2005,10,Lot 10,2018-01-01,2022-07-17 01:33:14,Monday,2018-01-01,2022-07-17 05:03:29,Monday,0,0,0 days,1,5.0,4.0
6743020,9056422,23050,70,NW (With Permit),2018-01-01,2022-07-17 01:14:17,Monday,NaT,NaT,NaN,0,1,NaT,1,NaN,NaN


In [48]:
# Bringing the lot capacity into the vtrans data frame
   
Left_join = pd.merge(vtrans, 
                     lotdata, 
                     on ='LotNumber', 
                     how ='left')
Left_join

,tTransID,CardNumber,LotNumber,EffectiveGroupName,EntranceDate,EntranceTime,EntranceDay,ExitDate,ExitTime,ExitDay,NoEntry,noexit,ParkedDays,EntryHour,ExistHour,ParkedTime,capacity
0,22840709,12777,13,NaN,2022-07-11,2022-07-17 12:29:09,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN,170.0
1,22840716,47951,40,NaN,2022-07-11,2022-07-17 12:29:09,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN,465.0
2,22840719,58760,3,NaN,2022-07-11,2022-07-17 12:29:04,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN,217.0
3,22840717,57369,20,NaN,2022-07-11,2022-07-17 12:29:02,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN,239.0
4,22840710,13947,11,NaN,2022-07-11,2022-07-17 12:28:53,Monday,NaT,NaT,NaN,0,1,NaT,12,NaN,NaN,591.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743017,9008592,22348,70,NW (With Permit),2018-01-01,2022-07-17 02:47:54,Monday,2018-01-01,2022-07-17 15:09:22,Monday,0,0,0 days,2,15.0,13.0,8.0
6743018,9007839,40907,70,NW ONLY,2018-01-01,2022-07-17 01:53:11,Monday,2018-01-01,2022-07-17 02:43:54,Monday,0,0,0 days,1,2.0,1.0,8.0
6743019,9007851,2005,10,Lot 10,2018-01-01,2022-07-17 01:33:14,Monday,2018-01-01,2022-07-17 05:03:29,Monday,0,0,0 days,1,5.0,4.0,462.0
6743020,9056422,23050,70,NW (With Permit),2018-01-01,2022-07-17 01:14:17,Monday,NaT,NaT,NaN,0,1,NaT,1,NaN,NaN,8.0


In [ ]:
# We could try a "pseudo" capacity utilization. We could assume each lot capacity in hours is 24 (hrs) x capacity 
# for any given day
# If we can groupby an aggregate function of LotNumber, EntranceDate and ParkedTime we could divide this to the 24 (hrs) x capacity
# does anyone know how to do this aggregation ?
